# collect all data and put them in a single dataframe

In [ ]:
import os
import pandas as pd

results_path = "/home/boonstra/24_04_11_with_checks"

data_frames = []
for filename in os.listdir(results_path):
    seed, extension = os.path.splitext(filename)
    if extension.lower() != ".csv":
        continue
    filepath = os.path.join(results_path, filename)
    df = pd.read_csv(filepath, header=0)
    df["seed"] = seed
    data_frames.append(df)

all_data = pd.concat(data_frames)

all_data.columns
from hofss import Task, TaskType, Factor, Scenario

hofs = Factor.parse_from_file("data/hofs_frequencies_and_multipliers.csv")
task_types = TaskType.parse_from_file("data/gtt_nhep_hofs.csv", hofs)
scenarios = Scenario.parse_from_file("data/scenarios.csv")
tasks = Task.parse_from_file("data/tasks.csv", task_types, scenarios)

typed_data = []
all_data["task_type"] = None
for task, group_data in all_data.groupby("task"):
    task_instance: Task = next(filter(lambda x: task == x.name, tasks), None)
    group_data["task_type"] = task_instance.task_type.name
    typed_data.append(group_data)
all_data = pd.concat(typed_data)
all_data.index = range(len(all_data))

In [ ]:
import numpy as np
scenario_data = all_data[all_data["scenario"].notna()].copy()
scenario_data = scenario_data[scenario_data["error_corrected"] != True]
previous_failure_probabilities = all_data.iloc[scenario_data.index - 1]["total"]
scenario_data["failure_probability_change"] = scenario_data["total"].values / previous_failure_probabilities.values
# scenario_data.dropna(inplace=True, subset=["failure_probability_change"])
scenario_data = scenario_data[~scenario_data["failure_probability_change"].isin([np.nan, np.inf, -np.inf])]


In [ ]:
def gmean(a):
    return np.prod(a)**(1/len(a))

def gmean_contributions(a: pd.Series):
    # remove nan_values
    b = a.dropna()

    contributors = []
    for index in b.index:
        contributors.append(gmean(b.values) / gmean(b.drop(index)))

    # return new series
    return pd.Series({k.replace("multiplier", "contribution"): v for k, v in zip(b.index, contributors)})

rows = []
multiplier_columns = [c for c in scenario_data.columns if "multiplier" in c]
count = 0
for row_index, row in scenario_data.iterrows():
    rows.append(pd.concat([row, gmean_contributions(row[multiplier_columns])]))
    count += 1

enriched_data = pd.concat(rows, axis=1).T
improvement_data = enriched_data[enriched_data["failure_probability_change"] <= 1.0]
degradation_data = enriched_data[enriched_data["failure_probability_change"] > 1.0]


In [ ]:
task_type_order = [tt.name for tt in task_types]
factors = [f"F{i}" for i in range(1,15)]

plot_values = []
plot_labels = []
for task_type in task_type_order:
    if task_type == "T14":
        continue
    task_type_values = []
    task_type_labels = []
    subset = enriched_data[enriched_data["task_type"] == task_type]
    probabilities = np.array(subset["failure_probability_change"].values, dtype=np.float64)

    for factor in factors:
        contributions = np.array(subset[f"{factor}_contribution"].values, dtype=np.float64)
        task_type_values.append(np.log10(np.mean(probabilities**contributions)))
        task_type_labels.append(np.mean(contributions))
    plot_values.append(task_type_values)
    plot_labels.append(task_type_labels)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.cm as cm

fig, ax = plt.subplots()
scale_factor = 3.0
fig.set_size_inches(w=80*scale_factor/25.4,h=60*scale_factor/25.4)
im = ax.imshow(plot_values,cmap=cm.get_cmap("OrRd"))
# im = ax.imshow(plot_values,cmap=cm.get_cmap("Greens_r"))

# Show all ticks and label them with the respective list entries
ax.set_xticks(np.arange(len(factors)), labels=[f.description for f in hofs])
ax.set_yticks(np.arange(len(task_type_order) - 1), labels=[tt.description for tt in task_types[:-1]])
ax.set_xlabel("Factor")
ax.set_ylabel("Task type")

# Rotate the tick labels and set their alignment.
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")

# Loop over data dimensions and create text annotations.
for i in range(len(task_type_order) - 1):
    for j in range(len(factors)):
        if np.isnan(plot_values[i][j]):
            continue
        text = ax.text(j, i, round(plot_values[i][j], 1), ha="center", va="center")

ax.set_title("HOFs effects structural failure probability")
shift=0.05
fig.tight_layout(rect=(shift,0,1+shift,1))
fig.savefig("all_sensitivity.png")
plt.show()
